In [1]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  60622      0 --:--:-- --:--:-- --:--:-- 60589
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http:/

In [2]:
# start server
# This code have to be executed before you can use ollama from the shell for some reason
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [3]:
# verify model availbility
!ollama list

NAME    ID    SIZE    MODIFIED 


In [4]:
# pull deepseek-r1:70b, this is the biggest one an A100 40GB can do, and it will max out the vram
# !ollama pull deepseek-r1:70b

In [5]:
# pull deepseek-r1:32b
!ollama pull deepseek-r1:32b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest 
pulling 6150cb382311...   0% ▕▏    0 B/ 19 GB                  pulling manifest 
pulling 6150cb382311...   0% ▕▏ 2.5 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   0% ▕▏  47 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   1% ▕▏ 146 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   1% ▕▏ 254 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   2% ▕▏ 309 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   2% ▕▏ 422 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   3% ▕▏ 535 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   3% ▕▏ 591 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   4% ▕▏ 707 MB/ 19 GB                  pulling manifest 
pulling 6150cb382311...   4% ▕▏ 813 MB/ 19 GB  759 MB/s   

In [6]:
!pip install ollama

In [7]:
import ollama
response = ollama.chat(model='deepseek-r1:32b', messages=[
  {
    'role': 'user',
    'content': "hello, how's your day?",
  },
])
print(response['message']['content'])

<think>

</think>

Hello! I don't have feelings, but I'm here and ready to help you with whatever you need. How's your day going? 😊


In [8]:
import re
import ollama
import json

# This code is a back and forth ChatGPT style implementation, and it also filter out the think tag in the output to keep the context short
def remove_think_sections(text):
    """
    Remove any content wrapped in <think>...</think> tags.
    This uses a regular expression to find and remove the internal reasoning.
    """
    # re.DOTALL makes sure newlines are included in '.'
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

# Initialize the conversation history
conversation = []

print("Welcome to the interactive chat with deepseek-r1!")
print("Type your message and hit Enter (type 'exit' to quit).\n")

while True:
    # Get the user's input
    user_input = input("User: ")
    if user_input.strip().lower() == 'exit':
        print("Exiting chat. Bye!")
        break

    # Append the new user message into the conversation history
    conversation.append({
        'role': 'user',
        'content': user_input
    })

    # Prepare the conversation context to be sent to the model
    # Remove <think> sections from any assistant messages
    conversation_for_model = []
    for msg in conversation:
        if msg['role'] == 'assistant':
            filtered_content = remove_think_sections(msg['content'])
            conversation_for_model.append({
                'role': msg['role'],
                'content': filtered_content
            })
        else:
            conversation_for_model.append(msg)

    # Request a response from the model with streaming enabled.
    response = ollama.chat(model='deepseek-r1:32b', messages=conversation_for_model, stream=True)

    model_response = ""
    print("Assistant: ", end="", flush=True)
    for chunk in response:
        content = chunk['message']['content']
        print(content, end="", flush=True)
        model_response += content
    print("\n")  # Newline after finishing the assistant's reply

    # Save the assistant's full response (including any <think> tags, if present) in the conversation
    conversation.append({
        'role': 'assistant',
        'content': model_response
    })

    print(conversation_for_model)

# After exiting the loop, save the complete conversation to a file.
output_file = 'conversation.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(conversation, f, ensure_ascii=False, indent=2)

print(f"\nConversation saved to {output_file}")

Welcome to the interactive chat with deepseek-r1!
Type your message and hit Enter (type 'exit' to quit).

User: please show me step by step how to do leetcode 200 in java
Assistant: <think>
Okay, I need to solve LeetCode problem 200, which is about finding the number of islands. Let's see what the problem says.

So, the task is given a grid of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and consists of connected land cells. Cells are connected if they are adjacent horizontally or vertically—so diagonals don't count.

First, I need to think about how to approach this problem. It seems like a classic graph traversal problem where each island can be thought of as a connected component in an undirected graph. So, maybe using Depth-First Search (DFS) or Breadth-First Search (BFS) would work here.

Let me outline the steps I need to take:

1. Iterate through each cell in the grid.
2. When I find a '1' that hasn't been visited yet, it's the star